In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 17176259
paper_name = 'bishop_avery_2007' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [7]:
files = ['hits_genenames_resistant.txt','hits_genenames_sensitive.txt']

In [13]:
original_data_list = []
for ixf, f in enumerate(files):
    original_data = pd.read_csv('raw_data/'+f, sep='\t', header=None)
    print('Original data dimensions: %d x %d' % (original_data.shape))
    print(original_data.head())
    original_data['gene'] = original_data[0].astype(str)
    original_data['gene'] = clean_genename(original_data['gene'])
    original_data['orf'] = translate_sc(original_data['gene'], to='orf')
    t = looks_like_orf(original_data['orf'])
    print(original_data.loc[~t,])
    
    original_data['data'] = np.power(-1, ixf)
    original_data.set_index('orf', inplace=True)
    original_data = original_data[['data']].copy()
    original_data = original_data.groupby(original_data.index).mean()
    
    print(original_data.shape)
    original_data_list.append(original_data)

Original data dimensions: 45 x 1
       0
0   HSL7
1   SIT4
2   DOA4
3  VPS24
4   VPS1
Empty DataFrame
Columns: [0, gene, orf]
Index: []
(34, 1)
Original data dimensions: 47 x 1
      0
0  CYC8
1  DOT6
2  GLO3
3  SSN8
4  EAF7
Empty DataFrame
Columns: [0, gene, orf]
Index: []
(47, 1)


In [14]:
original_data = pd.concat(original_data_list, axis=0)

In [15]:
original_data.shape

(81, 1)

In [16]:
original_data.head()

,data
orf,
YBR119W,1
YBR120C,1
YBR132C,1
YBR133C,1
YBR145W,1


In [17]:
original_data = original_data.groupby(original_data.index).mean()

In [18]:
original_data.shape

(81, 1)

# Load & process tested strains

In [22]:
tested1 = pd.read_excel('raw_data/Mata_DeletionArray+slow_growers.xlsx', sheet_name='96', skiprows=1)
tested2 = pd.read_excel('raw_data/Mata_DeletionArray+slow_growers.xlsx', sheet_name='slow growers', skiprows=1)

In [24]:
tested1.head()

,Gene,ORF,Plate,Row,Column,Unnamed: 5,Unnamed: 6,Unnamed: 7,Gene.1,ORF .1,...,ORF .7,Plate.7,Row.7,Column.7,Unnamed: 50,Gene.8,ORF .8,Plate.8,Row.8,Column.8
0,HIS3,YOR202W,1A,A,1,NaN,NaN,NaN,HIS3,YOR202W,...,YOR202W,10A,G,1.0,NaN,HIS3,YOR202W,10A,H,1.0
1,HIS3,YOR202W,1B,A,1,NaN,NaN,NaN,HIS3,YOR202W,...,YMR018W,10A,G,2.0,NaN,NaN,YMR031C,10A,H,2.0
2,HIS3,YOR202W,1A,A,2,NaN,NaN,NaN,HIS3,YOR202W,...,YMR017W,10A,G,3.0,NaN,RSF1,YMR030W,10A,H,3.0
3,HIS3,YOR202W,1B,A,2,NaN,NaN,NaN,HIS3,YOR202W,...,YMR016C,10A,G,4.0,NaN,FAR8,YMR029C,10A,H,4.0
4,HIS3,YOR202W,1A,A,3,NaN,NaN,NaN,HIS3,YOR202W,...,YMR015C,10A,G,5.0,NaN,HRT2,YMR027W,10A,H,5.0


In [34]:
tested = pd.concat([tested1[['ORF ']].rename(columns={'ORF ': 'ORF'}), tested2[['ORF']]], axis=0)

In [35]:
tested['orf'] = tested['ORF'].astype(str)

In [36]:
tested['orf'] = clean_orf(tested['orf'])

In [37]:
tested['orf'] = translate_sc(tested['orf'].values, to='orf')

In [38]:
# Make sure everything translated ok
t = looks_like_orf(tested['orf'])
print(tested.loc[~t,])

Empty DataFrame
Columns: [ORF, orf]
Index: []


In [40]:
tested_orfs = tested['orf'].unique()

In [41]:
missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
missing

['YCR038C', 'YHR051W', 'YHR091C', 'YNL220W', 'YOR147W']

In [42]:
tested_orfs = list(tested_orfs) + missing

In [43]:
original_data = original_data.reindex(index=tested_orfs, fill_value=0)

# Prepare the final dataset

In [44]:
data = original_data.copy()

In [45]:
dataset_ids = [173]
datasets = datasets.reindex(index=dataset_ids)

In [46]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [47]:
data.head()

dataset_id,173
data_type,value
orf,
YOR202W,0
YBR030W,0
YBR138C,0
YBR028C,0
YBR137W,0


## Subset to the genes currently in SGD

In [48]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [49]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,173
,data_type,value
gene_id,orf,
5414,YOR202W,0
226,YBR030W,0
332,YBR138C,0
224,YBR028C,0
331,YBR137W,0


# Normalize

In [50]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [51]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [52]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id        173          
data_type       value    valuez
gene_id orf                    
5414    YOR202W     0  0.076694
226     YBR030W     0  0.076694
332     YBR138C     0  0.076694
224     YBR028C     0  0.076694
331     YBR137W     0  0.076694

# Print out

In [53]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [54]:
from IO.save_data_to_db3 import *

In [55]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 17176259...
Inserting the new data...


100%|██████████| 1/1 [00:07<00:00,  7.97s/it]

Updating the data_modified_on field...
